In [1]:
from nanome.api.structure import Complex
import os
import time
import nglview as nv

# PDB file and fpocket results
pdb_file = "test_data/5ceo.pdb"
fpocket_out = 'test_data/5ceo_fpocket_out'


In [6]:
# Lets view a particular pocket number in NGLView
view = nv.show_structure_file(pdb_file)
view.representations = [
    {"type": "cartoon", "params": {
        "sele": "protein", "color": "blue"
    }},
    {"type": "ball+stick", "params": {
        "sele": "hetero"
    }}
]
view.add_representation("ball+stick", selection="LIG")


In [7]:
# Lets view a specific pocket number from the output
def view_pocket(view, fpocket_out, pocket_num):
    pocket_file = f'{fpocket_out}/pockets/pocket{pocket_num}_atm.pdb'
    if not os.path.exists(pocket_file):
        print(f"Pocket {pocket_num} not found")
        return False
    view.remove_surface()
    pocket_comp = Complex.io.from_pdb(path=pocket_file)
    pocket_res_nums = sorted([res.serial for res in pocket_comp.residues])
    view.add_surface(selection=pocket_res_nums, color='red', opacity=0.1)
    return True

pocket_number = 1
view_pocket(view, fpocket_out, pocket_number)
view

NGLWidget()

In [8]:
def cycle_pocket_surfaces(view, fpocket_out: str):
    # Clear all other surface representations
    i = 1
    while True:
        view.remove_surface()
        pocket_file = f'{fpocket_out}/pockets/pocket{i}_atm.pdb'
        if not os.path.exists(pocket_file):
            break
        success = view_pocket(view, fpocket_out, i)
        if not success:
            break
        print(f"Pocket {i}")
        time.sleep(1)
        i += 1

view
cycle_pocket_surfaces(view, fpocket_out)

Pocket 1
Pocket 2
Pocket 3
Pocket 4
Pocket 5
Pocket 6
Pocket 7
Pocket 8
Pocket 9
Pocket 10
Pocket 11
Pocket 12
Pocket 13
Pocket 14


In [ ]:
# Set up nanome plugin
import os
from nanome.beta.redis_interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = os.environ.get("REDIS_HOST")
redis_port = os.environ.get("REDIS_PORT")
redis_password = os.environ.get("REDIS_PASSWORD")
redis_channel = ""  # Use room id code you receive when activating cookbook plugin

plugin = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
plugin.connect()
print('Plugin Connected')

In [ ]:
plugin.send_files_to_load([pdb_file])

In [ ]:
comp = plugin.request_workspace().complexes[-1]

In [ ]:
from nanome.util import Color
from nanome.api.structure import Complex

def render_pocket_surface(comp, fpocket_out, pocket_num):
    pocket_file = f'{fpocket_out}/pockets/pocket{pocket_num}_atm.pdb'
    if not os.path.exists(pocket_file):
        print(f"Pocket {pocket_num} not found")
        return False
    pocket_comp = Complex.io.from_pdb(path=pocket_file)
    pocket_res_nums = sorted([res.serial for res in pocket_comp.residues])
    for res in comp.residues:
        surface_color = Color.Red() if res.serial in pocket_res_nums else Color.Blue()
        for atom in res.atoms:
            atom.surface_rendering = True
            atom.surface_color = surface_color
    comp._surface_dirty = True

pocket_number = 5
render_pocket_surface(comp, fpocket_out, pocket_number)
plugin.update_structures_deep([comp])
